In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
import numpy as np
import open3d as o3d
from rlbench.action_modes.action_mode import MoveArmThenGripper
from rlbench.action_modes.arm_action_modes import JointVelocity
from rlbench.action_modes.gripper_action_modes import Discrete
from rlbench.environment import Environment
from rlbench.tasks import ReachTarget

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
action_mode = MoveArmThenGripper(
  arm_action_mode=JointVelocity(),
  gripper_action_mode=Discrete()
)
env = Environment(action_mode, headless=False)
env.launch()

task = env.get_task(ReachTarget)
descriptions, obs = task.reset()
obs, reward, terminate = task.step(np.random.normal(size=env.action_shape))

In [2]:
import sys
sys.path.append('../')

from src.rlbench.enviroment import RLBenchEnv

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
import numpy as np
action_bounds = np.array([-0.3, -0.5, 0.6]), np.array([0.7, 0.5, 1.6])
nbins = 10

In [4]:
env = RLBenchEnv('ReachTarget', action_bounds, nbins)

In [6]:
trajs = env.get_demos(20)

Keyframes num:  2
Keyframes num:  3
Keyframes num:  3
Keyframes num:  3
Keyframes num:  4
Keyframes num:  3
Keyframes num:  3
Keyframes num:  3
Keyframes num:  2
Keyframes num:  2
Keyframes num:  2
Keyframes num:  2
Keyframes num:  4
Keyframes num:  2
Keyframes num:  2
Keyframes num:  2
Keyframes num:  3
Keyframes num:  2
Keyframes num:  3
Keyframes num:  3


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../')

import jax
import jax.numpy as jnp
jax.config.update('jax_platform_name', 'cpu')
from flax import linen as nn
key = jax.random.PRNGKey(0)

In [3]:
from src.types_ import Observation
from src.config import Config
from src.networks import Networks

In [8]:
obs = Observation(voxels=jnp.zeros((1, 7, 7, 7, 4)), low_dim=jnp.zeros((1, 34)))
cfg = Config()

In [5]:
model = Networks(cfg)

In [6]:
params = model.init(key, obs)

> /home/khromykhla/Documents/github/perciever/src/networks.py(226)__call__()
    224     def __call__(self, obs: types.Observation) -> tfd.Distribution:
    225         import pdb; pdb.set_trace()
--> 226         x = self.preprocessor(obs)
    227         x = self.encoder(x)
    228         logits = nn.Dense(8 * self.config.nbins)(x)



ipdb>  n


> /home/khromykhla/Documents/github/perciever/src/networks.py(227)__call__()
    225         import pdb; pdb.set_trace()
    226         x = self.preprocessor(obs)
--> 227         x = self.encoder(x)
    228         logits = nn.Dense(8 * self.config.nbins)(x)
    229         logits = logits.reshape((8, self.config.nbins))



ipdb>  n


> /home/khromykhla/Documents/github/perciever/src/networks.py(228)__call__()
    226         x = self.preprocessor(obs)
    227         x = self.encoder(x)
--> 228         logits = nn.Dense(8 * self.config.nbins)(x)
    229         logits = logits.reshape((8, self.config.nbins))
    230         dist = tfd.Categorical(logits)



...

ipdb>  n


> /home/khromykhla/Documents/github/perciever/src/networks.py(230)__call__()
    227         x = self.encoder(x)
    228         logits = nn.Dense(8 * self.config.nbins)(x)
    229         logits = logits.reshape((8, self.config.nbins))
--> 230         dist = tfd.Categorical(logits)
    231         return tfd.Independent(dist, 1)



ipdb>  n


> /home/khromykhla/Documents/github/perciever/src/networks.py(231)__call__()
    227         x = self.encoder(x)
    228         logits = nn.Dense(8 * self.config.nbins)(x)
    229         logits = logits.reshape((8, self.config.nbins))
    230         dist = tfd.Categorical(logits)
--> 231         return tfd.Independent(dist, 1)



ipdb>  c


In [9]:
out = model.apply(params, obs)

> /home/khromykhla/Documents/github/perciever/src/networks.py(226)__call__()
    224     def __call__(self, obs: types.Observation) -> tfd.Distribution:
    225         import pdb; pdb.set_trace()
--> 226         x = self.preprocessor(obs)
    227         x = self.encoder(x)
    228         logits = nn.Dense(8 * self.config.nbins)(x)



ipdb>  
ipdb>  c


In [11]:
out.sample(seed=key)

Array([3, 1, 5, 3, 9, 7, 0, 9], dtype=int32)

In [12]:
from rlbench import tasks